In [1]:
import os
import geopandas
import logging
import pandas as pd
import io
import subprocess
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.patheffects as path_effects
import folium
from folium.plugins import HeatMap

try:
    from PT3S import dxAndMxHelperFcts
except:
    import dxAndMxHelperFcts
try:
    from PT3S import Rm
except:
    import Rm
try:
    from PT3S import ncd
except:
    import ncd

In [2]:
import logging

logger = logging.getLogger()

logFileName = r"pNcd_DH.log"

loglevel = logging.DEBUG
logging.basicConfig(
    filename=logFileName,
    filemode='w',
    level=loglevel,
    format="%(asctime)s ; %(name)-60s ; %(levelname)-7s ; %(message)s ; [%(filename)s:%(lineno)d - %(funcName)s]"
)

fileHandler = logging.FileHandler(logFileName)
logger.addHandler(fileHandler)

consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(logging.Formatter("%(levelname)-7s ; %(message)s ; [%(filename)s:%(lineno)d - %(funcName)s]"))
consoleHandler.setLevel(logging.INFO)
logger.addHandler(consoleHandler)


In [23]:
dbFilename="Example5"
dbFile=os.path.join(os.path.dirname(os.path.abspath(dxAndMxHelperFcts.__file__))
                    +'/Examples/'
                    +dbFilename
                    +'.db3'
)

In [24]:
m=dxAndMxHelperFcts.readDxAndMx(dbFile=dbFile                                 
                                ,preventPklDump=True

)
#NBVAL_IGNORE_OUTPUT

INFO    ; Dx.__init__: dbFile (abspath): c:\users\aUserName\3s\pt3s\Examples\Example5.db3 exists readable ... ; [Dx.py:322 - __init__]
INFO    ; dxAndMxHelperFcts.readDxAndMx: 
+..\Examples\Example5.db3 is newer than
+..\Examples\WDExample5\B1\V0\BZ1\M-1-0-1.1.MX1:
+SIR 3S' dbFile is newer than SIR 3S' mx1File
+in this case the results are maybe dated or (worse) incompatible to the model ; [dxAndMxHelperFcts.py:1948 - readDxAndMx]
INFO    ; dxAndMxHelperFcts.readDxAndMx: 
+..\Examples\WDExample5\B1\V0\BZ1\M-1-0-1.XML is newer than
+..\Examples\WDExample5\B1\V0\BZ1\M-1-0-1.1.MX1:
+SirCalc's xmlFile is newer than SIR 3S' mx1File
+in this case the results are maybe dated or (worse) incompatible to the model ; [dxAndMxHelperFcts.py:1960 - readDxAndMx]
INFO    ; Mx.setResultsToMxsFile: Mxs: ..\Examples\WDExample5\B1\V0\BZ1\M-1-0-1.1.MXS reading ... ; [Mx.py:2156 - setResultsToMxsFile]
INFO    ; dxWithMx.__init__: Example5: processing dx and mx ... ; [dxAndMxHelperFcts.py:115 - __init__]
ERR

In [6]:
df=m.V3_KNOT

In [12]:
df.dtypes

pk                                                                    object
fkDE                                                                  object
rk                                                                    object
tk                                                                    object
NAME                                                                  object
                                                                      ...   
(TIME, KNOT~*~*~*~VOLD, 2020-01-28 14:45:01, 2020-01-28 14:45:01)    float64
(TMIN, KNOT~*~*~*~VOLD, 2020-01-28 14:45:01, 2020-01-28 14:46:01)    float64
(TMAX, KNOT~*~*~*~VOLD, 2020-01-28 14:45:01, 2020-01-28 14:46:01)    float64
PH                                                                   float64
dPH                                                                   object
Length: 201, dtype: object

In [38]:
# Assuming df is your DataFrame

# Find columns that do not start with '('
filtered_cols = [col for col in df.columns if not col.startswith('(')]

# Filter the DataFrame to include only these columns
filtered_df = df[filtered_cols]

# Display the filtered DataFrame
print(filtered_cols)


['pk', 'fkDE', 'rk', 'tk', 'NAME', 'KTYP', 'XKOR', 'YKOR', 'ZKOR', 'QM_EIN', 'LFAKT', 'fkPZON', 'fkFSTF', 'fkUTMP', 'fkFQPS', 'fkCONT', 'fk2LKNOT', 'BESCHREIBUNG', 'GEOMWKB', 'DELETED', 'SELECT1', 'IDREFERENZ', 'KENNUNG', 'IPLANUNG', 'KVR', 'DRAKONZ', 'pk_BZ', 'fkDE_BZ', 'fk', 'fkPVAR', 'fkQVAR', 'fkLFKT', 'PH_EIN', 'TM', 'TE', 'PH_MIN', 'GRAF', 'MId', 'MText', 'Basis', 'Variante', 'BZ', 'Geaendert', 'Erstellt', 'pk_VMBZ', 'fkBASIS', 'fkVARIANTE', 'fkBZ', 'MZ', 'MT', 'pk_CONT', 'fkDE_CONT', 'rk_CONT', 'tk_CONT', 'ID_CONT', 'NAME_CONT', 'IDPARENT_CONT', 'rkPARENT_CONT', 'LFDNR_CONT', 'GRAF_CONT', 'FONT_CONT', 'GEOM_CONT', 'DELETED_CONT', 'SELECT1_CONT', 'IDREFERENZ_CONT', 'pk_VKNO', 'fkDE_VKNO', 'rk_VKNO', 'fkKNOT_VKNO', 'fkCONT_VKNO', 'SHOWNAME_VKNO', 'POSNAME_VKNO', 'POINTINSERTX_VKNO', 'POINTINSERTY_VKNO', 'SYMBOLFACT_VKNO', 'DISPLAYMODE_VKNO', 'DELETED_VKNO', 'SELECT1_VKNO', 'MId_VKNO', 'MText_VKNO', 'Basis_VKNO', 'Variante_VKNO', 'BZ_VKNO', 'Geaendert_VKNO', 'Erstellt_VKNO', 'pk_VM

In [14]:
# Assuming df is your DataFrame
import re

# Ensure all column names are strings
df.columns = df.columns.map(str)

# Find columns starting with 'qs_' and 'qs'
qsColsEgr = [col for col in df.columns if re.search('^qs_', col)]
qsColsInf = [col for col in df.columns if re.search('^qs', col) and not re.search('^qs_', col)]
qsCols = qsColsEgr + qsColsInf

# Print the columns
print("Columns starting with 'qs':", qsCols)


Columns starting with 'qs': []


In [26]:
m.dx.dataFrames['EBES_BZ']

,pk,fkDE,fk,AKTIV,AKTIVQS,VERSAGENSW,GRAF
0,-1,None,None,NaN,NaN,NaN,None
1,5184693889865717961,5032285329913031496,5622504138093320653,100.0,101.0,0.0,None
2,5011706316339838135,5032285329913031496,5408410878987506184,101.0,101.0,0.0,None


In [27]:
m.dx.dataFrames['EBES']

,pk,fkDE,rk,tk,NAME,LFDNR,OBJS,DELETED,SELECT1,IDREFERENZ
0,-1,None,None,None,None,NaN,None,NaN,None,None
1,5622504138093320653,5613149064237404433,5622504138093320653,5601957102971980247,B,2.0,b'FWES~5175187672733343279',0.0,None,-1
2,5408410878987506184,5613149064237404433,5408410878987506184,4947747250455344469,A,1.0,b'FWES~5237816858835684263',0.0,None,-1


In [28]:
EBES = m.dx.dataFrames['EBES']
EBES_BZ = m.dx.dataFrames['EBES_BZ']
vEBES=pd.merge(EBES,EBES_BZ,left_on='pk',right_on='fk',suffixes=('','_BZ')).sort_values(by=['LFDNR','AKTIVQS']).reset_index()

In [29]:
vEBES

,index,pk,fkDE,rk,tk,NAME,LFDNR,OBJS,DELETED,SELECT1,IDREFERENZ,pk_BZ,fkDE_BZ,fk,AKTIV,AKTIVQS,VERSAGENSW,GRAF
0,1,5408410878987506184,5613149064237404433,5408410878987506184,4947747250455344469,A,1.0,b'FWES~5237816858835684263',0.0,None,-1,5011706316339838135,5032285329913031496,5408410878987506184,101.0,101.0,0.0,None
1,0,5622504138093320653,5613149064237404433,5622504138093320653,5601957102971980247,B,2.0,b'FWES~5175187672733343279',0.0,None,-1,5184693889865717961,5032285329913031496,5622504138093320653,100.0,101.0,0.0,None


In [30]:
Names=vEBES['NAME'].tolist()
Lfdnrs=vEBES['LFDNR'].tolist()
expColNames=['qs' + '_' + str(Lfdnr) + '_' + Name for Lfdnr,Name in zip(Lfdnrs,Names)]

In [31]:
expColNames


['qs_1.0_A', 'qs_2.0_B']

In [32]:
vKNOTexp=m.V3_KNOT    
vKNOTexp['KVR'].fillna('0',inplace=True) # vermeiden, dass spaetere Aggregierungen mit KVR Null-Ergebnisse produzieren für KVR-Nulls

if 'KNOT~*~*~*~ESQUELLSP' in vKNOTexp.columns.tolist():
    vKNOTexp['qsStr']=vKNOTexp['KNOT~*~*~*~ESQUELLSP'].str.decode('utf-8')
    vKNOTexp['qsStr']=vKNOTexp['qsStr'].str.rstrip()

    # die Anzahl der Spalten ist die Anzahl der definierten ### berechneten Ebes
    expDf=vKNOTexp['qsStr'].str.split('\t', expand = True)

    # die Spalten dranhängen (heißen 1,2,...)
    vKNOTexp=pd.merge(vKNOTexp,expDf,left_index=True,right_index=True,suffixes=('','_expDf'))

In [34]:
print(vKNOTexp.columns.tolist)

<bound method IndexOpsMixin.tolist of Index([                                                                      'pk',
                                                                           'fkDE',
                                                                             'rk',
                                                                             'tk',
                                                                           'NAME',
                                                                           'KTYP',
                                                                           'XKOR',
                                                                           'YKOR',
                                                                           'ZKOR',
                                                                         'QM_EIN',
       ...
          ('STAT', 'KNOT~*~*~*~PH_EIN', 2024-01-09 23:00:00, 2024-01-09 23:00:00),
          ('STAT', 'KNOT~*~*~*~PH_MIN'